# Chatbot Modeling

## Import Libs

In [69]:
# General
import json
import string
import pickle
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Deep Learning
import tensorflow as tf

# Word Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # Only needed for first time usage of NLTK

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Files for Preprocessing and Modeling

### Intent

In [70]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [71]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [72]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


### Stopwords

In [73]:
# Define stemmer and stopword
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [74]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

### Slang

In [75]:
# slang = pd.read_csv('lexicon/slang ke semi baku.csv')

# slang_replace = {}
# for i, row in enumerate(slang['slang']):
#     slang_replace[row] = slang['formal'].iloc[i]

In [76]:
# baku = pd.read_csv('lexicon/slang ke baku.csv')

# std_word_replace = {}
# for i, row in enumerate(baku['slang']):
#     std_word_replace[row] = baku['baku'].iloc[i]

In [77]:
baku = pd.read_csv('lexicon/baku.csv')

In [78]:
std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

In [79]:
std_word_replace['gak']

'tidak'

## Preprocessing

In [80]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in std_word_replace: 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [81]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [82]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

## Create Corpus

In [83]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [84]:
corpus_size = len(words)

In [85]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [86]:
sequence_length = int(round(df['length'].max(),0))

In [87]:
corpus_size, sequence_length

(358, 8)

## Label Encoding

In [88]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = tf.keras.utils.to_categorical(y_train)

In [89]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [90]:
len(le.classes_)

22

## Modeling

### Text Vectorization

In [91]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [92]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([23, 12, 11,  0,  0,  0,  0,  0], dtype=int64)>

### Word Embedding

In [93]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [94]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[ 0.03853462, -0.03360705,  0.02211213, -0.01795112, -0.03602955,
         0.04949274,  0.0031283 ,  0.03274949, -0.02107451,  0.02702581,
        -0.03820109,  0.0433121 ,  0.00571074, -0.01101174,  0.01575703,
         0.0268165 ],
       [ 0.02885098, -0.03947011,  0.01563777, -0.02747667, -0.03002098,
        -0.00772154,  0.00211558, -0.00292844,  0.00856432,  0.0006563 ,
        -0.0308546 ,  0.02973196, -0.01230348, -0.02144375, -0.02638157,
        -0.03389976],
       [ 0.02081262, -0.04149457,  0.04172282,  0.02582288, -0.02856763,
        -0.01862653,  0.00862879,  0.03750468,  0.04291407, -0.00556409,
         0.02454365,  0.01561158,  0.01999387,  0.03208322, -0.04166334,
         0.01679399],
       [-0.04628502,  0.00121652, -0.02749063, -0.01016929,  0.04322923,
        -0.01837881, -0.01531589,  0.02788894,  0.02633411,  0.00433249,
        -0.00319986,  0.01254359, -0.04987613, -0.00223448,  0.02571288,
         

### Layering

In [95]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

### Training

In [96]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.019533,1.0
96,0.019120,1.0
97,0.018803,1.0
98,0.018251,1.0
99,0.017757,1.0


### Evaluating Model

In [97]:
model.evaluate(df['clean_input'], y_train)

19/19 [==============================] - 4s 4ms/step - loss: 0.0174 - categorical_accuracy: 1.0000


[0.01736104115843773, 1.0]

In [98]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        13
           biaya       1.00      1.00      1.00        28
          daftar       1.00      1.00      1.00        22
         dokumen       1.00      1.00      1.00        23
    error upload       1.00      1.00      1.00        15
             ktp       1.00      1.00      1.00        18
     link sosmed       1.00      1.00      1.00        45
        lowongan       1.00      1.00      1.00        22
   lupa password       1.00      1.00      1.00        24
            nama       1.00      1.00      1.00        10
        pas foto       1.00      1.00      1.00        23
       pengantar       1.00      1.00      1.00        17
         penutup       1.00      1.00      1.00        50
   qualification       1.00      1.00      1.00        26
responsibilities       1.00      1.00      1.00        35
          salary       1.00      1.00      1.00        38
            s

### Function Covering End-to-End Process

In [99]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()] # Get the index with highest probability
    i = 0
    try:
        if textvect(text).numpy().max() > 1: # If the input is known word(s)
            while i < len(intent_json['intents']):
                if res == intent_json['intents'][i]['intent']:
                    response = intent_json['intents'][i]['response']
                    break
                else:
                    i+=1
        else: # If only unknown word(s)
            response = ['Maaf, Kak. Aku tidak mengerti...']
    except: # If empty string or any error occured
        response = ['Maaf, Kak. Aku tidak mengerti...']

    # For debugging only
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])

    return print(np.random.choice(response))

In [100]:
tes1 = 'aku mau daftar rekrutmen'
clean_text(tes1)

'aku mau daftar rekrutmen'

In [101]:
model.predict([clean_text(tes1)])

array([[2.08448342e-04, 1.09997904e-06, 9.89931822e-01, 2.39695466e-04,
        8.31362093e-04, 2.15991793e-04, 2.99815838e-05, 5.90342861e-05,
        1.36565045e-03, 1.17290610e-05, 1.74880624e-04, 3.64732332e-05,
        4.49598127e-04, 1.51047728e-03, 9.22848049e-05, 1.06580956e-05,
        3.40960505e-05, 5.65641074e-07, 4.04407829e-03, 1.08255745e-05,
        8.88806198e-07, 7.40399002e-04]], dtype=float32)

In [102]:
model.predict([clean_text(tes1)]).argmax()

2

In [103]:
le.classes_[model.predict([clean_text(tes1)]).argmax()]

'daftar'

In [104]:
le.classes_[model.predict([clean_text(tes1)]).argmax()] == intent_json['intents'][2]['intent']

True

In [105]:
intent_json['intents'][2]['response']

['Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:\n1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara\n2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi\n3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di https://rekrutmen.fiktif.id/news atau email masing-masing']

In [106]:
bot_response('aku mau daftar rekrutmen')

[{'CV': 0.00020844834}, {'biaya': 1.099979e-06}, {'daftar': 0.9899318}, {'dokumen': 0.00023969547}, {'error upload': 0.0008313621}, {'ktp': 0.0002159918}, {'link sosmed': 2.9981584e-05}, {'lowongan': 5.9034286e-05}, {'lupa password': 0.0013656504}, {'nama': 1.1729061e-05}, {'pas foto': 0.00017488062}, {'pengantar': 3.6473233e-05}, {'penutup': 0.00044959813}, {'qualification': 0.0015104773}, {'responsibilities': 9.2284805e-05}, {'salary': 1.0658096e-05}, {'sapa': 3.409605e-05}, {'sertifikat': 5.656411e-07}, {'skck': 0.0040440783}, {'timeline': 1.08255745e-05}, {'training': 8.888062e-07}, {'transkrip-ijazah': 0.000740399}]
daftar
Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:
1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara
2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi
3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutn

### Saving Model

In [107]:
# pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [108]:
pickle.dump({'config': textvect.get_config(),
             'weights': textvect.get_weights()}
            , open("saved_model/textvect.pkl", "wb"))

In [109]:
model.save('saved_model/model', save_traces=True)

INFO:tensorflow:Assets written to: saved_model/model\assets


INFO:tensorflow:Assets written to: saved_model/model\assets


## Inference

In [110]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([279,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [111]:
clean_text('ok')

'ok'

In [112]:
textvect('ok').numpy().max() > 1

True

In [113]:
bot_response('a')

[{'CV': 0.00014926693}, {'biaya': 0.0009817937}, {'daftar': 0.0029506683}, {'dokumen': 0.004454021}, {'error upload': 1.0452405e-05}, {'ktp': 1.5375803e-06}, {'link sosmed': 0.0049550817}, {'lowongan': 0.0005463395}, {'lupa password': 0.009204468}, {'nama': 2.383838e-05}, {'pas foto': 0.018794892}, {'pengantar': 5.5192544e-05}, {'penutup': 0.4635196}, {'qualification': 0.0005873692}, {'responsibilities': 4.0081006e-05}, {'salary': 0.024214271}, {'sapa': 0.005021482}, {'sertifikat': 1.5208798e-05}, {'skck': 5.2372266e-06}, {'timeline': 0.45977518}, {'training': 0.0006664098}, {'transkrip-ijazah': 0.0040275985}]
penutup
Maaf, Kak. Aku tidak mengerti...


In [114]:
bot_response('ya')

[{'CV': 4.8645627e-05}, {'biaya': 9.297839e-05}, {'daftar': 8.39374e-05}, {'dokumen': 0.00059122563}, {'error upload': 1.2777886e-06}, {'ktp': 2.6287344e-08}, {'link sosmed': 0.0010752971}, {'lowongan': 1.7060756e-05}, {'lupa password': 0.00033030257}, {'nama': 1.5115894e-06}, {'pas foto': 0.00044017186}, {'pengantar': 3.206031e-06}, {'penutup': 0.9921267}, {'qualification': 0.00028312803}, {'responsibilities': 3.4790216e-06}, {'salary': 0.00067263364}, {'sapa': 0.0012251198}, {'sertifikat': 2.5750117e-07}, {'skck': 2.912159e-06}, {'timeline': 0.0028858148}, {'training': 2.262961e-05}, {'transkrip-ijazah': 9.157889e-05}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [115]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [116]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [117]:
bot_response('oh ok')

[{'CV': 3.94773e-05}, {'biaya': 1.8849532e-05}, {'daftar': 0.0007545028}, {'dokumen': 0.01639448}, {'error upload': 5.526689e-06}, {'ktp': 4.024321e-07}, {'link sosmed': 0.0007895099}, {'lowongan': 4.71152e-06}, {'lupa password': 0.0027690588}, {'nama': 5.6055637e-06}, {'pas foto': 0.0004012197}, {'pengantar': 2.196414e-06}, {'penutup': 0.9726863}, {'qualification': 0.0044182516}, {'responsibilities': 2.1075653e-05}, {'salary': 0.00034652805}, {'sapa': 0.00042660194}, {'sertifikat': 8.289796e-08}, {'skck': 7.781334e-05}, {'timeline': 0.0004984812}, {'training': 4.7245862e-06}, {'transkrip-ijazah': 0.00033458098}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [118]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 0.0013804277}, {'biaya': 0.0006941786}, {'daftar': 0.00023378422}, {'dokumen': 2.6587766e-05}, {'error upload': 0.012864039}, {'ktp': 0.00032194174}, {'link sosmed': 0.0056037037}, {'lowongan': 0.83321065}, {'lupa password': 6.057517e-07}, {'nama': 1.613094e-05}, {'pas foto': 9.318026e-05}, {'pengantar': 0.13507889}, {'penutup': 1.0898733e-05}, {'qualification': 0.00030367819}, {'responsibilities': 0.006136759}, {'salary': 0.001077325}, {'sapa': 0.00030397807}, {'sertifikat': 0.0016794958}, {'skck': 7.488825e-06}, {'timeline': 2.3474164e-05}, {'training': 0.0009315298}, {'transkrip-ijazah': 1.2752273e-06}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [119]:
bot_response('ingin')

[{'CV': 0.003968113}, {'biaya': 0.024110552}, {'daftar': 0.00024694053}, {'dokumen': 0.00066814624}, {'error upload': 8.429675e-05}, {'ktp': 5.4167863e-06}, {'link sosmed': 0.3084902}, {'lowongan': 0.017921079}, {'lupa password': 0.00015259824}, {'nama': 0.0006130161}, {'pas foto': 0.0074404273}, {'pengantar': 0.0002531179}, {'penutup': 0.36954764}, {'qualification': 0.013508225}, {'responsibilities': 0.0019993146}, {'salary': 0.011219099}, {'sapa': 0.22163841}, {'sertifikat': 5.5981087e-05}, {'skck': 0.00010695672}, {'timeline': 0.0056106234}, {'training': 0.011911318}, {'transkrip-ijazah': 0.0004485653}]
penutup
Maaf, Kak. Aku tidak mengerti...


In [120]:
bot_response('pas foto hilang gimana ya')

[{'CV': 7.264111e-05}, {'biaya': 8.040791e-05}, {'daftar': 0.00020830285}, {'dokumen': 1.7837606e-06}, {'error upload': 0.00029116304}, {'ktp': 9.902271e-07}, {'link sosmed': 2.4861673e-05}, {'lowongan': 0.00029221067}, {'lupa password': 0.0017457632}, {'nama': 4.1985393e-05}, {'pas foto': 0.99150956}, {'pengantar': 3.6541098e-06}, {'penutup': 0.00243599}, {'qualification': 1.523724e-05}, {'responsibilities': 4.391056e-07}, {'salary': 0.000763338}, {'sapa': 9.460553e-05}, {'sertifikat': 4.352439e-06}, {'skck': 2.5162421e-06}, {'timeline': 0.00033102836}, {'training': 0.0017707733}, {'transkrip-ijazah': 0.00030833704}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format file 

ktp, pas foto, password masih ketuker2

In [121]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [122]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([114,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [123]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [124]:
data = {'user_input':'hai hai'}
data = json.dumps(data)
loaded_data = json.loads(data)

In [125]:
type(loaded_data)

dict

In [126]:
tespred = str(model.predict([clean_text(loaded_data['user_input'])]).argmax())

In [127]:
tes_res = {'pred':tespred}
tes_res = json.dumps(tes_res)
loaded_res = json.loads(tes_res)

In [128]:
le.classes_[int(loaded_res['pred'])]

'sapa'

In [129]:
loaded_data['user_input']

'hai hai'

In [130]:
1000

1000

In [131]:
str(1000)

'1000'

In [132]:
int(str(1000))

1000

In [133]:
le.classes_[int(str(10))]

'pas foto'

In [134]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([114,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [135]:
user_input = 'apa kualifikasi engineer'
clean_input = clean_text(user_input)
pred = model.predict([clean_input]).argmax()
if textvect(clean_input).numpy().max() <= 1:
    label_idx = 1000
else:
    label_idx = pred 
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']

print(np.random.choice(response))

Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:
1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan
2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.
3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun
4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL
5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik


In [136]:
le.classes_[label_idx]

'qualification'